Pitch columns:
'px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir',
       'break_angle', 'break_length', 'break_y', 'ax', 'ay', 'az', 'sz_bot',
       'sz_top', 'type_confidence', 'vx0', 'vy0', 'vz0', 'x', 'x0', 'y', 'y0',
       'z0', 'pfx_x', 'pfx_z', 'nasty', 'zone', 'code', 'type', 'pitch_type',
       'event_num', 'b_score', 'ab_id', 'b_count', 's_count', 'outs',
       'pitch_num', 'on_1b', 'on_2b', 'on_3b']

Game columns: 'attendance', 'away_final_score', 'away_team', 'date',
       'elapsed_time', 'g_id', 'home_final_score', 'home_team', 'start_time',
       'umpire_1B', 'umpire_2B', 'umpire_3B', 'umpire_HP', 'venue_name',
       'weather', 'wind', 'delay'

At bats columns: 'inning', 'top', 'ab_id', 'g_id', 'p_score', 'batter_id',
       'pitcher_id', 'stand', 'p_throws', 'event', 'o'

Player columns: id, first_name, last_name

### Topics to Discuss w/ Instructor (3/8):
- General workflow for pitch classification - model for each pitcher individually?
- Clustering hitters as a feature  -is pulling stats potential data leak (since those are pitches in the dataset)  
- PItch arsenal - data leak?
- How to best divide train/validation and test sets (2015-2018 train/val, 2019 test?)

### Discussion Topics - 3/16  
- Workflow of modeling validating/testing, on each individual pitcher?  
- More correct way of regression to get coordinates? (i.e. predictions basically flip shape when reversed)  
- Regression/classification: standard scaling of count featres? (Inning, outs, strikes, balls, etc.)

# SQL Queries:

In [ ]:
query = '''
--Queuing up the game/at bat info from teh above query:
WITH game_player_ab AS (

WITH game_ab AS (
SELECT a.inning, a.batter_id, a.pitcher_id, a.top, a.ab_id, a.p_score, a.stand, a.p_throws, a.event, g.date, g.home_team, g.away_team
FROM atbats as a
RIGHT JOIN games as g
    ON a.g_id = g.g_id
)

SELECT g.*, 
p.first_name as Pitcher_First_Name, p.last_name as Pitcher_Last_Name, 
h.first_name as Hitter_First_Name, h.last_name as Hitter_Last_Name

FROM game_ab as g

--first, joining up the pitcher's name:
LEFT JOIN players as p
    ON g.pitcher_id = p.id

--now, joining the hitter's name:
LEFT JOIN players as h
    ON g.batter_id = h.id

--Ordering:
ORDER BY g.ab_id ASC
)

SELECT 
--First, taking all the data above, along with the batting team score:
gpa.*, pi.b_score,

--Adding the runners on base:
pi.on_1b, pi.on_2b, pi.on_3b,

--Now, selecting data from the pitch table:
pi.px, pi.pz, pi.pitch_type, pi.start_speed,pi.type, pi.b_count, pi.s_count, pi.outs, pi.pitch_num,
pi.type_confidence, pi.nasty, pi.code

--Finally, adding in previous pitch info (for that at bat).  This will lead to some NaN values for the first pitch of an at-bat, which is expected:
(max(pi.pitch_type) OVER (PARTITION BY gpa.ab_id ORDER BY pi.pitch_num ASC ROWS 1 PRECEDING EXCLUDE CURRENT ROW)) as last_pitch_type,
(max(pi.px) OVER (PARTITION BY gpa.ab_id ORDER BY pi.pitch_num ASC ROWS 1 PRECEDING EXCLUDE CURRENT ROW)) as last_pitch_px,
(max(pi.pz) OVER (PARTITION BY gpa.ab_id ORDER BY pi.pitch_num ASC ROWS 1 PRECEDING EXCLUDE CURRENT ROW)) as last_pitch_pz,
(max(pi.start_speed) OVER (PARTITION BY gpa.ab_id ORDER BY pi.pitch_num ASC ROWS 1 PRECEDING EXCLUDE CURRENT ROW)) as last_pitch_speed

FROM pitches as pi
RIGHT JOIN game_player_ab as gpa
    ON pi.ab_id = gpa.ab_id
    
--Ordering:
ORDER BY gpa.ab_id ASC, pi.pitch_num ASC
LIMIT 10;
;
'''
combined_data_df = pd.read_sql(query, engine)

combined_data_df.head(10)